In [1]:
import time
notebook_start_time = time.time()

# Coding Attention Mechanism

## Simple Self-Attention Without Trainable Weights

Let's create a simple example:

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

inputs.shape # [token, embedding_dim]

torch.Size([6, 3])

In [3]:
query_2 = inputs[1] # 2nd input token is the query
attention_scores_2 = torch.empty(inputs.shape[0])

# compute attention scores
for i, input_embedding in enumerate(inputs):
  attention_scores_2[i] = torch.dot(query_2, input_embedding)

attention_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [4]:
# compute attention weights
attention_weights_2 = torch.softmax(attention_scores_2, dim=0)

print("attention weights: ", attention_weights_2)
print("sum:", attention_weights_2.sum())

attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum: tensor(1.)


In [5]:
# compute context vector
context_vector_2 = torch.zeros(inputs.shape[1])

for i, input_embedding in enumerate(inputs):
  context_vector_2 += attention_weights_2[i] * input_embedding

context_vector_2

tensor([0.4419, 0.6515, 0.5683])

Let's compute for all queries simultaneously using matrices:

In [6]:
# compute attention scores
attention_scores = inputs @ inputs.T
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [7]:
# compute attention weights
attention_weights = torch.softmax(attention_scores, dim=-1)
print("row sums:", attention_weights.sum(dim=-1))
attention_weights

row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [8]:
# compute context vectors
context_vectors = attention_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Implement Self-Attention with Trainable Weights

## Compute KVQ vectors


In [9]:
# define some variables
x_2 = inputs[1] # 2nd input token is the query
input_embedding_dim = inputs.shape[1] # dim=3
output_embedding_dim = 5 # output dim=2

Let's create KVQ matrices:

In [10]:
torch.manual_seed(211)
W_query = torch.nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim),
                             requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim),
                           requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim),
                             requires_grad=False)

In [11]:
# project x_2 into KVQ spaces
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print("query_2:", query_2)
print("key_2:", key_2)
print("value_2:", value_2)

query_2: tensor([0.3530, 0.8074, 0.7720, 0.9246, 0.7118])
key_2: tensor([0.6885, 0.4568, 1.4283, 1.4383, 1.2989])
value_2: tensor([1.2123, 0.6055, 1.0735, 1.3861, 1.5435])


Let's compute all the projected keys and values:

In [12]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys:", keys)
print("values:", values)

keys: tensor([[0.4207, 0.2279, 1.2477, 0.9477, 0.7168],
        [0.6885, 0.4568, 1.4283, 1.4383, 1.2989],
        [0.6925, 0.4572, 1.4219, 1.4353, 1.2789],
        [0.3541, 0.2496, 0.7168, 0.7615, 0.7592],
        [0.5700, 0.3358, 0.9077, 0.9768, 0.5587],
        [0.3412, 0.2621, 0.8338, 0.8443, 0.9977]])
values: tensor([[0.6973, 0.4836, 1.1020, 1.0985, 0.9538],
        [1.2123, 0.6055, 1.0735, 1.3861, 1.5435],
        [1.2069, 0.6000, 1.0771, 1.3826, 1.5277],
        [0.6661, 0.3190, 0.4886, 0.7061, 0.8630],
        [0.7701, 0.3332, 0.8395, 0.9301, 0.8138],
        [0.7621, 0.3945, 0.4978, 0.7934, 1.0709]])


In [13]:
# compute the attention scores of x_2
attention_scores_2 = query_2 @ keys.T
attention_scores_2

tensor([2.6821, 3.9688, 3.9486, 2.1243, 2.4738, 2.4665])

In [14]:
# compute attention weights
keys_embedding_dim = keys.shape[-1]
attention_weights_2 = torch.softmax(attention_scores_2 / keys_embedding_dim**0.5, dim=-1)
attention_weights_2

tensor([0.1401, 0.2491, 0.2468, 0.1092, 0.1276, 0.1272])

In [15]:
# compute context vectors
context_vector_2 = attention_weights_2 @ values
context_vector_2

tensor([0.9655, 0.4942, 0.9114, 1.1371, 1.2295])

## Implement a Self-Attention Class

In [16]:
import torch.nn as nn

class SelfAttentionV1(nn.Module):
  def __init__(self, input_embedding_dim, output_embedding_dim):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim))
    self.W_key = nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim))
    self.W_value = nn.Parameter(torch.rand(input_embedding_dim, output_embedding_dim))

  def forward(self, inputs):
    keys = inputs @ self.W_key
    values = inputs @ self.W_value
    queries = inputs @ self.W_query

    attention_scores = queries @ keys.T
    attention_weights = torch.softmax(
        attention_scores / keys.shape[-1]**0.5, dim=-1)
    context_vectors = attention_weights @ values
    return context_vectors

In [17]:
torch.manual_seed(211)
sa_v1 = SelfAttentionV1(input_embedding_dim, output_embedding_dim)
context_vectors = sa_v1(inputs)
context_vectors

tensor([[0.9414, 0.4835, 0.8960, 1.1131, 1.1991],
        [0.9655, 0.4942, 0.9114, 1.1371, 1.2295],
        [0.9641, 0.4936, 0.9103, 1.1357, 1.2278],
        [0.9298, 0.4771, 0.8825, 1.0980, 1.1843],
        [0.9173, 0.4713, 0.8726, 1.0845, 1.1688],
        [0.9458, 0.4847, 0.8955, 1.1156, 1.2045]], grad_fn=<MmBackward0>)

## Implement a Self-Attention Class With Linear Layers

In [18]:
class SelfAttentionV2(nn.Module):
  def __init__(self, input_embedding_dim, output_embedding_dim, qkv_bias=False):
    super().__init__()
    self.W_query = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_key = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_value = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)

  def forward(self, inputs):
    keys = self.W_key(inputs)
    values = self.W_value(inputs)
    queries = self.W_query(inputs)

    attention_scores = queries @ keys.T
    attention_weights = torch.softmax(
        attention_scores / keys.shape[-1]**0.5, dim=-1)
    context_vectors = attention_weights @ values
    return context_vectors

In [19]:
torch.manual_seed(211)
sa_v2 = SelfAttentionV2(input_embedding_dim, output_embedding_dim)
context_vectors = sa_v2(inputs)
context_vectors

tensor([[ 0.2783,  0.4747, -0.4724, -0.0341,  0.3074],
        [ 0.2768,  0.4746, -0.4740, -0.0356,  0.3078],
        [ 0.2768,  0.4747, -0.4741, -0.0353,  0.3078],
        [ 0.2798,  0.4775, -0.4724, -0.0288,  0.3086],
        [ 0.2779,  0.4780, -0.4733, -0.0269,  0.3090],
        [ 0.2799,  0.4764, -0.4724, -0.0316,  0.3081]], grad_fn=<MmBackward0>)

## Causal Attention Mask

In [20]:
# setup
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attention_scores = queries @ keys.T
attention_scores

tensor([[-0.1403, -0.0855, -0.1058,  0.0263, -0.4402,  0.2156],
        [-0.1708, -0.0352, -0.0575,  0.0688, -0.4432,  0.2793],
        [-0.1734, -0.0347, -0.0566,  0.0683, -0.4349,  0.2748],
        [-0.0777, -0.0100, -0.0220,  0.0403, -0.2321,  0.1540],
        [-0.1706, -0.0153, -0.0234,  0.0407, -0.1629,  0.1161],
        [-0.0599, -0.0171, -0.0340,  0.0472, -0.3295,  0.2079]],
       grad_fn=<MmBackward0>)

In [21]:
# create the lower triangle matrix of 1's
context_length = inputs.shape[0]
masked_matrix = torch.triu(torch.ones(context_length, context_length),
                           diagonal=1
                           )
print(masked_matrix)
print(masked_matrix.bool())

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])


In [22]:
# perform masked attention: fill upper triangle of attention_scores with -inf
masked_attention_scores = attention_scores.masked_fill(
    masked_matrix.bool(), -torch.inf)
masked_attention_scores

tensor([[-0.1403,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.1708, -0.0352,    -inf,    -inf,    -inf,    -inf],
        [-0.1734, -0.0347, -0.0566,    -inf,    -inf,    -inf],
        [-0.0777, -0.0100, -0.0220,  0.0403,    -inf,    -inf],
        [-0.1706, -0.0153, -0.0234,  0.0407, -0.1629,    -inf],
        [-0.0599, -0.0171, -0.0340,  0.0472, -0.3295,  0.2079]],
       grad_fn=<MaskedFillBackward0>)

In [23]:
# compute masked attention weights
masked_attention_weights = torch.softmax(
    masked_attention_scores / keys.shape[-1]**0.5, dim=-1)
masked_attention_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4848, 0.5152, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3208, 0.3413, 0.3380, 0.0000, 0.0000, 0.0000],
        [0.2433, 0.2508, 0.2494, 0.2565, 0.0000, 0.0000],
        [0.1907, 0.2045, 0.2037, 0.2097, 0.1914, 0.0000],
        [0.1641, 0.1673, 0.1660, 0.1722, 0.1455, 0.1850]],
       grad_fn=<SoftmaxBackward0>)

## Masked Attention Weights + Dropout

In [24]:
torch.manual_seed(211)
dropout_05 = torch.nn.Dropout(p=0.5)
dropout_05(masked_attention_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9697, 1.0303, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6415, 0.6826, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4989, 0.5130, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4075, 0.4193, 0.0000, 0.0000],
        [0.3282, 0.3346, 0.3320, 0.3443, 0.0000, 0.3700]],
       grad_fn=<MulBackward0>)

## Implement a Causual Attention Class

In [25]:
# batch setup
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

torch.Size([2, 6, 3])

In [26]:
class CausalAttention(nn.Module):
  def __init__(self,
               input_embedding_dim,
               output_embedding_dim,
               context_length,
               dropout,
               qkv_bias=False):
    super().__init__()
    self.output_embedding_dim = output_embedding_dim
    self.W_query = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_key = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_value = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer(
        "mask",
        torch.triu(torch.ones(context_length, context_length),
                                            diagonal=1))

  def forward(self, inputs):
    batch, num_tokens, input_embedding_dim = inputs.shape
    keys = self.W_key(inputs)
    values = self.W_value(inputs)
    queries = self.W_query(inputs)

    attention_scores = queries @ keys.transpose(1, 2)
    attention_scores.masked_fill_(
        self.mask.bool()[:num_tokens, :num_tokens], - torch.inf)
    masked_attention_weight = torch.softmax(
        attention_scores / (keys.shape[-1]**0.5),
        dim=-1)
    masked_attention_dropout_weight = self.dropout(masked_attention_weight)

    context_vector = masked_attention_dropout_weight @ values
    return context_vector

In [27]:
# create an instance
torch.manual_seed(211)
context_length = batch.shape[1]
ca = CausalAttention(input_embedding_dim=input_embedding_dim,
                     output_embedding_dim=output_embedding_dim,
                     context_length=context_length,
                     dropout=0.2)
context_vectors = ca(batch)
print("context_vector shape: ", context_vectors.shape)

context_vector shape:  torch.Size([2, 6, 5])


## Multi-Head Attention

### Create a Wrapper Class to Implement Multi-Head Attention (SEQUENTIALLY)

In [28]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self,
               input_embedding_dim,
               output_embedding_dim,
               context_length,
               dropout,
               num_heads,
               qkv_bias=False):
    super().__init__()
    self.heads = nn.ModuleList(
        [CausalAttention(input_embedding_dim,
                         output_embedding_dim,
                         context_length,
                         dropout,
                         qkv_bias)
            for _ in range(num_heads)
        ]
    )

  def forward(self, inputs):
    return torch.cat(
        [heads(inputs) for heads in self.heads], # process SEQUENTIALLY
        dim=-1)

In [29]:
# variable setup
torch.manual_seed(211)
context_length = batch.shape[1]
input_embedding_dim = 3
output_embedding_dim = 5


# create an instance
mha = MultiHeadAttentionWrapper(
    input_embedding_dim,
    output_embedding_dim,
    context_length,
    dropout=0.2,
    num_heads=2
)
context_vectors = mha(batch)

print("context_vectors shape: ", context_vectors.shape)
print("context_vectors: \n\n", context_vectors)

context_vectors shape:  torch.Size([2, 6, 10])
context_vectors: 

 tensor([[[ 0.7202,  0.5802, -0.3213,  0.0145,  0.3137,  0.5136,  0.5068,
           0.6836,  0.4716, -0.4601],
         [ 0.5586,  0.6915, -0.5847, -0.0264,  0.4228,  0.4128,  0.0706,
           0.3255,  0.1374, -0.3347],
         [ 0.3679,  0.4561, -0.3822, -0.0102,  0.2780,  0.5537,  0.0995,
           0.4337,  0.1837, -0.4539],
         [ 0.2402,  0.5018, -0.5408, -0.0536,  0.3349,  0.3344,  0.0324,
           0.2566,  0.0996, -0.2617],
         [ 0.3334,  0.5203, -0.5027, -0.0410,  0.3332,  0.6177,  0.2268,
           0.5055,  0.2518, -0.5587],
         [ 0.2826,  0.4628, -0.4534, -0.0323,  0.2980,  0.6232,  0.1607,
           0.5190,  0.2407, -0.5171]],

        [[ 0.7202,  0.5802, -0.3213,  0.0145,  0.3137,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.5586,  0.6915, -0.5847, -0.0264,  0.4228,  0.6798,  0.3340,
           0.6809,  0.3825, -0.5738],
         [ 0.3679,  0.4561, -0.3822, -0.010

Implement Multi-Head Attention With Weight Splits (PARALLELLY)

In [30]:
class MultiHeadAttention(nn.Module):
  def __init__(self,
               input_embedding_dim,
               output_embedding_dim,
               context_length,
               dropout,
               num_heads,
               qkv_bias=False):
    super().__init__()
    assert (output_embedding_dim % num_heads == 0), \
          "output_embedding_dim must be divisible by num_heads"

    self.output_embedding_dim = output_embedding_dim
    self.num_heads = num_heads
    self.head_dim = output_embedding_dim // num_heads
    self.W_query = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_key = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.W_value = nn.Linear(input_embedding_dim, output_embedding_dim,
                             bias=qkv_bias)
    self.output_projection = nn.Linear(output_embedding_dim,
                                       output_embedding_dim) # to combine head outputs
    self.dropout = nn.Dropout(dropout)
    self.register_buffer(
        "mask",
        torch.triu(torch.ones(context_length, context_length),
                                            diagonal=1))

  def forward(self, inputs):
    batch, num_tokens, input_embedding_dim = inputs.shape

    # qkv shapes : (batch, num_tokens, output_embedding_dim)
    keys = self.W_key(inputs)
    values = self.W_value(inputs)
    queries = self.W_query(inputs)

    # qkv shapes : (batch, num_tokens, num_heads, head_dim)
    keys = keys.view(batch, num_tokens, self.num_heads, self.head_dim)
    values = values.view(batch, num_tokens, self.num_heads, self.head_dim)
    queries = queries.view(batch, num_tokens, self.num_heads, self.head_dim)

    # qkv shapes : (batch, num_heads, num_tokens, head_dim)
    keys = keys.transpose(1, 2)
    values = values.transpose(1, 2)
    queries = queries.transpose(1, 2)

    # compute attention scores for each head
    attention_scores = queries @ keys.transpose(3, 2)
    attention_scores.masked_fill(
        self.mask.bool()[:num_tokens, :num_tokens], - torch.inf)

    # compute attention weights + dropout
    masked_attention_weight = torch.softmax(
        attention_scores / (keys.shape[-1]**0.5),
        dim=-1)
    masked_attention_dropout_weight = self.dropout(masked_attention_weight)

    # compute context vectors
    # shape : (batch, num_tokens, num_heads, head_dim)
    context_vector = (masked_attention_dropout_weight @ values).transpose(1, 2)

    # combine heads, where self.d_out = self.num_heads * self.head_dim
    # shape : (batch, num_tokens, output_embedding_dim)
    context_vector = context_vector.contiguous().view(
        batch, num_tokens, self.output_embedding_dim)

    # linear projection (optional)
    context_vector = self.output_projection(context_vector)

    return context_vector

In [31]:
# variable setup
torch.manual_seed(211)
print(batch.shape)
batch_size, context_length, input_embedding_dim = batch.shape
output_embedding_dim = 2


# create an instance
mha = MultiHeadAttentionWrapper(
    input_embedding_dim,
    output_embedding_dim,
    context_length,
    dropout=0.2,
    num_heads=3
)
context_vectors = mha(batch)

print("context_vectors shape: ", context_vectors.shape)
print("context_vectors: \n\n", context_vectors)

torch.Size([2, 6, 3])
context_vectors shape:  torch.Size([2, 6, 6])
context_vectors: 

 tensor([[[-0.1217,  0.6112,  0.7202,  0.5802, -0.1637, -0.5053],
         [-0.2210,  0.4563,  0.5639,  0.6879,  0.0155, -0.3606],
         [-0.2433,  0.4075,  0.5112,  0.7247, -0.0413, -0.6645],
         [-0.2400,  0.3400,  0.3825,  0.5433, -0.0060, -0.5926],
         [-0.1524,  0.2967,  0.2414,  0.5206, -0.0074, -0.4772],
         [-0.1859,  0.2038,  0.2334,  0.5033,  0.0194, -0.2693]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.2210,  0.4563,  0.5639,  0.6879, -0.0694, -0.6227],
         [-0.1442,  0.2974,  0.5112,  0.7247,  0.0000,  0.0000],
         [-0.1672,  0.2592,  0.1380,  0.2996, -0.0060, -0.5926],
         [-0.1342,  0.2008,  0.2775,  0.4027, -0.0591, -0.5935],
         [-0.0740,  0.1984,  0.3534,  0.5999, -0.0239, -0.3668]]],
       grad_fn=<CatBackward0>)


## Some Note On `.contiguous()`

> A tensor is **contiguous** if its data is stored in memory in row-major (C-style) order — meaning the next element in a row is stored next to the current one in memory. Think of it as how the tensor is laid out in memory, not how it looks visually.



In [32]:
import torch

# Create a 2x3 tensor
x = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])
print("Original shape:", x.shape)
print("Is contiguous?", x.is_contiguous())

Original shape: torch.Size([2, 3])
Is contiguous? True


In [33]:
# Transpose it
x_t = x.transpose(0, 1)
print(x_t)
print("Transposed shape:", x_t.shape)
print("Is contiguous?", x_t.is_contiguous())

tensor([[1, 4],
        [2, 5],
        [3, 6]])
Transposed shape: torch.Size([3, 2])
Is contiguous? False


In [34]:
print(x.stride())      # e.g., (3, 1) → row-major: step 3 for next row
print(x_t.stride())    # e.g., (1, 3) → shows it’s not row-major anymore

(3, 1)
(1, 3)


In [35]:
# Try to view it — this will raise an error if you don't call contiguous()
try:
    x_t.view(-1)
except RuntimeError as e:
    print("Error:", e)

Error: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.


In [36]:
# Fix with .contiguous()
x_fixed = x_t.contiguous().view(-1)
print("Fixed and reshaped:", x_fixed)

Fixed and reshaped: tensor([1, 4, 2, 5, 3, 6])


## Some Note On `output_projection()`

This linear layer:

- Mixes information across heads

- Allows the model to recombine features learned by each head

- Helps fuse multiple perspectives (heads) into a single output space

Without it:
- Each head contributes independently

- No opportunity to re-weight or mix features learned by other heads

- You would limit the representational power



In [37]:
notebook_end_time = time.time()
runtime_in_seconds = notebook_end_time - notebook_start_time

# format as minutes and seconds
minutes, seconds = divmod(runtime_in_seconds, 60)
print(f"Notebook runtime: {int(minutes)} min {seconds:.2f} sec")

Notebook runtime: 0 min 3.18 sec
